In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
anime = pd.read_csv('C:\\Users\\Mudit Nahata\\Desktop\\Python Internship\\anime_dataset\\anime.csv')
#rating = pd.read_csv('rating.csv')

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
anime['genre'].unique()

array(['Drama, Romance, School, Supernatural',
       'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen',
       'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen',
       ..., 'Hentai, Sports', 'Drama, Romance, School, Yuri',
       'Hentai, Slice of Life'], dtype=object)

In [5]:
#Fill number of episodes of OVA and Movie types to 1
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1" 

anime.loc[(anime["type"] == "Movie") & (anime["episodes"] == "Unknown")] = "1"

In [6]:
#Fill number of episodes of some other anime :

fill_anime = {"Naruto Shippuuden":500, "One Piece":925,"Detective Conan":956, "Dragon Ball Super":131,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}

for k,v in fill_anime.items():    
    anime.loc[anime["name"]==k,"episodes"] = v


In [7]:
#Replace values which are NaN with mean values


anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)
anime["episodes"] = anime["episodes"].astype(float)
anime["episodes"].fillna(anime["episodes"].mean(),inplace = True)

anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].mean(),inplace = True)

In [8]:
pd.get_dummies(anime[["type"]]).head()

,type_1,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


In [9]:
type(anime["members"][0])

int

In [10]:
anime["members"] = anime["members"].astype(float)

In [11]:
anime.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [12]:
#To make each genre as a feature
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),pd.get_dummies(anime[["type"]]),anime[["rating"]],anime[["members"]],anime["episodes"]],axis=1)

In [13]:
anime_features.columns

Index([' Adventure', ' Cars', ' Comedy', ' Dementia', ' Demons', ' Drama',
       ' Ecchi', ' Fantasy', ' Game', ' Harem', ' Hentai', ' Historical',
       ' Horror', ' Josei', ' Kids', ' Magic', ' Martial Arts', ' Mecha',
       ' Military', ' Music', ' Mystery', ' Parody', ' Police',
       ' Psychological', ' Romance', ' Samurai', ' School', ' Sci-Fi',
       ' Seinen', ' Shoujo', ' Shoujo Ai', ' Shounen', ' Shounen Ai',
       ' Slice of Life', ' Space', ' Sports', ' Super Power', ' Supernatural',
       ' Thriller', ' Vampire', ' Yaoi', ' Yuri', '1', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space',
       'Sports', 'Super Power', 'Supernat

In [14]:
#Scaling the data

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
anime_features = scale.fit_transform(anime_features)

C:\Users\Mudit Nahata\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Mudit Nahata\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [15]:
#KNN algorithm to find the 11 nearest neighbours, so as to recommend 10 other anime

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=11).fit(anime_features)
distances, indices = nbrs.kneighbors(anime_features)

In [16]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]

In [17]:
all_anime_names = list(anime.name.values)

In [18]:
#function to return 10 recommendations based on the input anime name/id

def get_recommendation_from_name(query):
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(anime.iloc[id]["name"])
            
def get_recommendation_from_id(num):
    for num in indices[num][1:]:
            print(anime.iloc[num]["name"])

In [21]:
get_recommendation_from_name("Death Note")

Higurashi no Naku Koro ni Kai
Death Note Rewrite
Mirai Nikki (TV)
Boku dake ga Inai Machi
Mahou Shoujo Madoka★Magica
Re:Zero kara Hajimeru Isekai Seikatsu
Ergo Proxy
Psycho-Pass
Shiki
Hyouka


In [20]:
get_recommendation_from_id(22)

Tengen Toppa Gurren Lagann
Seihou Bukyou Outlaw Star
Akame ga Kill!
D.Gray-man
Trigun
Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka
Hai to Gensou no Grimgar
Uchuu Patrol Luluco
Noragami
Guilty Crown
